In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def block1(inputs,filters):
  x=inputs
  for i in range(2):
    x=tf.keras.layers.Conv2D(filters=filters,kernel_size=(3,3),strides=(1,1),padding="same")(x)
  return x

In [ ]:
def block2(inputs,filters):
  x=inputs
  for i in range(3):
    x=tf.keras.layers.Conv2D(filters=filters,kernel_size=(3,3),strides=(1,1),padding="same")(x)
  return x

In [ ]:
def max_pooling1(inputs,filters):
  f=block1(inputs,filters)
  p=tf.keras.layers.MaxPooling2D(pool_size=(2,2),strides=(2,2))(f)
  return f,p

In [ ]:
def max_pooling2(inputs,filters):
  f=block2(inputs,filters)
  p=tf.keras.layers.MaxPooling2D(pool_size=(2,2),strides=(2,2))(f)
  return f,p

In [ ]:
def encoder1(inputs):
  f1,p1=max_pooling1(inputs,64)
  f2,p2=max_pooling1(p1,128)
  f3,p3=max_pooling2(p2,256)
  f4,p4=max_pooling2(p3,512)
  f5,p5=max_pooling2(p4,512)
  #print(p5.shape)
  return p5,(f1,f2,f3,f4,f5)

In [ ]:
def EBS(convs):
  f1,f2,f3,f4,f5=convs
  x1=tf.keras.layers.MaxPooling2D(pool_size=(8,8),strides=(8,8))(f1)
  x1=tf.keras.layers.Conv2D(filters=2,kernel_size=(1,1),strides=(1,1),padding="same")(x1)
  x2=tf.keras.layers.MaxPooling2D(pool_size=(4,4),strides=(4,4))(f2)
  x2=tf.keras.layers.Conv2D(filters=2,kernel_size=(1,1),strides=(1,1),padding="same")(x2)
  x3=tf.keras.layers.MaxPooling2D(pool_size=(2,2),strides=(2,2))(f3)
  x3=tf.keras.layers.Conv2D(filters=2,kernel_size=(1,1),strides=(1,1),padding="same")(x3)
  x4=tf.keras.layers.MaxPooling2D(pool_size=(2,2),strides=(2,2))(f4)
  x4=tf.keras.layers.Conv2D(filters=2,kernel_size=(1,1),strides=(1,1),padding="same")(x4)
  x5=tf.keras.layers.Conv2D(filters=2,kernel_size=(1,1),strides=(1,1),padding="same")(f5)
  o1=tf.keras.layers.Add()([x1,x2,x3])
  o2=tf.keras.layers.Add()([x4,x5])
  return o1,o2

In [ ]:
def RPE(p5):
  x1=tf.keras.layers.Conv2DTranspose(filters=6,kernel_size=(4,4),strides=(2,2),padding='same')(p5)
  x1=tf.keras.layers.Conv2DTranspose(filters=6,kernel_size=(4,4),strides=(2,2),padding='same')(x1)
  x1=tf.keras.layers.Conv2DTranspose(filters=6,kernel_size=(16,16),strides=(8,8),padding='same')(x1)
  return x1

In [ ]:
def enhanced_feature(o1,o2,p5):
  map1 = tf.keras.layers.Conv2DTranspose(filters=2,kernel_size=(4,4),strides=(2,2),padding='same')(p5)
  out1 = tf.keras.layers.Add()([o2,map1])
  map2 = tf.keras.layers.Conv2DTranspose(filters=2,kernel_size=(4,4),strides=(2,2),padding='same')(out1)
  out2 = tf.keras.layers.add(([o1,map2]))
  map3 = tf.keras.layers.Conv2DTranspose(filters=6,kernel_size=(16,16),strides=(8,8),padding='same')(out2)
  return map3

In [ ]:
from sklearn.cluster import MeanShift
#from keras import backend as K
import numpy as np
def mask(x1,map3):
  result=tf.bitwise.bitwise_and(tf.cast(x1, tf.int32),tf.cast(map3, tf.int32))
  print(result.shape)
  #result = tf.make_ndarray(result)
  clustering = MeanShift(bandwidth=2).fit(result)
  cluster = clustering.labels_
  return result

In [ ]:
def lanenet():
  inputs=tf.keras.layers.Input(shape=(224,224,3,))
  p,convs=encoder1(inputs)
  o1,o2=EBS(convs)
  rpe=RPE(p)
  map=enhanced_feature(o1,o2,p)
  #cluster=mask(rpe,map)
  model1=tf.keras.Model(inputs=inputs,outputs=map)
  model2=tf.keras.Model(inputs=inputs,outputs=rpe)
  model1.summary()
  model2.summary()
  return model1,model2

In [ ]:
lanenet()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 224, 224, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 224, 224, 64) 36928       conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 112, 112, 64) 0           conv2d_1[0][0]                   
______________________________________________________________________________________________

(<tensorflow.python.keras.engine.functional.Functional at 0x7fd5d0b33590>,
 <tensorflow.python.keras.engine.functional.Functional at 0x7fd5d0b1b7d0>)

In [ ]:
def main():
  model1,model2=lanenet()
  model1.compile(optimizer=tf.keras.optimizers.Adam(5e-4),loss='binary_crossentropy',metrics=['accuracy'])
  model2.compile(optimizer=tf.keras.optimizers.Adam(5e-4), loss=tf.keras.regularizers.L1L2(l1=0.0,l2=0.0),metrics=['accuracy'])
  model1.fit()
  model2.fit()
  model1.summary()
  model2.summary()

In [ ]:
tf.compat.v1.enable_eager_execution()